In [2]:
from random import sample
import random
from tqdm import tqdm
import pandas as pd
import requests
import time
import DHB as dhb
tqdm.pandas()

In [234]:
dhb.db_open()
df = dhb.sql("select * from match_aram")
dhb.db_close()

OPEN
close


In [29]:
df = pd.read_csv("725700.csv")

# 1. 스펠의 키값과 url 매칭

In [3]:
url = "https://ddragon.leagueoflegends.com/cdn/11.24.1/data/ko_KR/summoner.json"
res = requests.get(url).json()
res

{'type': 'summoner',
 'version': '11.24.1',
 'data': {'SummonerBarrier': {'id': 'SummonerBarrier',
   'name': '방어막',
   'description': '2초 동안 방어막으로 감싸 피해를 115~455(챔피언 레벨에 따라 변동)만큼 흡수합니다.',
   'tooltip': '2초간 방어막이 챔피언을 감싸 {{ f1 }}의 피해를 흡수합니다.',
   'maxrank': 1,
   'cooldown': [180],
   'cooldownBurn': '180',
   'cost': [0],
   'costBurn': '0',
   'datavalues': {},
   'effect': [None, [95], [20], [0], [0], [0], [0], [0], [0], [0], [0]],
   'effectBurn': [None, '95', '20', '0', '0', '0', '0', '0', '0', '0', '0'],
   'vars': [],
   'key': '21',
   'summonerLevel': 4,
   'modes': ['NEXUSBLITZ',
    'URF',
    'PRACTICETOOL',
    'TUTORIAL',
    'CLASSIC',
    'ARAM',
    'DOOMBOTSTEEMO',
    'ULTBOOK',
    'ONEFORALL',
    'ARSR',
    'ASSASSINATE',
    'FIRSTBLOOD',
    'PROJECT',
    'STARGUARDIAN'],
   'costType': '소모값 없음',
   'maxammo': '-1',
   'range': [1200],
   'rangeBurn': '1200',
   'image': {'full': 'SummonerBarrier.png',
    'sprite': 'spell0.png',
    'group': 'spell',
    'x':

In [34]:
res["data"]["Summoner_UltBookSmitePlaceholder"]["key"]

'55'

In [10]:
spellId = list(res["data"].keys())

In [14]:
spellkey = list(map(lambda x: res["data"][x]["key"], spellId))

In [35]:
spellurl = list(map(lambda x: "https://ddragon.leagueoflegends.com/cdn/11.24.1/img/spell/"+x+".png", spellId))

In [38]:
spell = dict(zip(spellkey,spellurl))

In [41]:
spell

{'21': 'https://ddragon.leagueoflegends.com/cdn/11.24.1/img/spell/SummonerBarrier.png',
 '1': 'https://ddragon.leagueoflegends.com/cdn/11.24.1/img/spell/SummonerBoost.png',
 '14': 'https://ddragon.leagueoflegends.com/cdn/11.24.1/img/spell/SummonerDot.png',
 '3': 'https://ddragon.leagueoflegends.com/cdn/11.24.1/img/spell/SummonerExhaust.png',
 '4': 'https://ddragon.leagueoflegends.com/cdn/11.24.1/img/spell/SummonerFlash.png',
 '6': 'https://ddragon.leagueoflegends.com/cdn/11.24.1/img/spell/SummonerHaste.png',
 '7': 'https://ddragon.leagueoflegends.com/cdn/11.24.1/img/spell/SummonerHeal.png',
 '13': 'https://ddragon.leagueoflegends.com/cdn/11.24.1/img/spell/SummonerMana.png',
 '30': 'https://ddragon.leagueoflegends.com/cdn/11.24.1/img/spell/SummonerPoroRecall.png',
 '31': 'https://ddragon.leagueoflegends.com/cdn/11.24.1/img/spell/SummonerPoroThrow.png',
 '11': 'https://ddragon.leagueoflegends.com/cdn/11.24.1/img/spell/SummonerSmite.png',
 '39': 'https://ddragon.leagueoflegends.com/cdn/11

# ----------------------------------------------------------------------------------------------------------

# 2. 각 챔피언마다 스펠의 픽률과 승률

In [30]:
df["WIN"] = df.apply(lambda x: 1 if x["WIN"] == True else 0 , axis=1)

In [167]:
def sepll_lst(row):
    tmp = [row["SPELL1"],row["SPELL2"]]
    tmp.sort(reverse=True)
    tmp = list(map(str,tmp))
    tmp = "|".join(tmp)
    return tmp

In [168]:
df["SEPLL_LST"] = df.apply(lambda x: sepll_lst(x), axis=1)

In [169]:
df

,GAMEID,PARTICIPANTID,CHAMPIONID,CHAMPIONNAME,SPELL1,SPELL2,MAIN_RUNE,SUB_RUNE,STAT_RUNE,TEAMID,...,TOTALTIMECCDEALT,TIMECCINGOTHERS,LARGESTKILLINGSPREE,GOLDEARNED,STARTING_ITEM,MAIN_ITEM_BUILD,SUB_ITEM_BUILD,BOOTS,SKILL_BUILD,SEPLL_LST
0,KR_5656757004,1,12,Alistar,4,32,8400|8439|8463|8473|8451,8300|8304|8347,5002|5003|5005,100,...,336,88,2,12546,1006|1006|1006|2051,3083|3001|3193,0,3111,1|2|3|1|1|4|1|2|1|2|4|2|2|3|3|4|3|3,32|4
1,KR_5656757004,2,222,Jinx,1,4,8000|8008|8009|9103|8017,8100|8139|8135,5002|5008|5005,100,...,56,15,2,13818,1001|2031|3184,6672|3085|3031,0,3006,1|2|3|1|1|4|1|2|1|2|4|2|2|3|3|4|3|3,4|1
2,KR_5656757004,3,267,Nami,4,3,8100|8112|8126|8138|8106,8000|8009|8014,5003|5002|5008,100,...,298,48,2,13061,2003|3070|3112,6617|3504|3011,3003,3158,1|2|3|2|2|4|2|3|2|3|4|1|1|3|3|4|1|1,4|3
3,KR_5656757004,4,268,Azir,4,1,8200|8229|8226|8210|8236,8000|8009|8014,5002|5008|5005,100,...,190,14,6,16570,3802,6653|3115|3135,3102|3165,3020,1|3|1|1|4|1|2|1|2|4|2|2|3|3|4|3|3,4|1
4,KR_5656757004,5,121,Khazix,32,4,8100|8128|8126|8138|8135,8000|8009|8014,5002|5008|5008,100,...,256,22,3,14117,3070|3177,6691|6693|3004,3142|6691,3111,1|2|3|2|2|4|2|2|1|2|1|4|1|1|1|3|3|4|3|3,32|4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326725,KR_5667335218,6,236,Lucian,3,4,8100|8128|8143|8138|8106,8000|9104|8009,5003|5008|5005,200,...,11,1,2,11389,3070|3177,6692|3004|6694,0,3158,1|2|3|1|1|4|1|3|1|3|4|3|3|2,4|3
326726,KR_5667335218,7,112,Viktor,1,4,8000|8021|8009|9105|8014,8200|8226|8210,5002|5008|5008,200,...,297,38,4,12880,2003|3070|3112,6655|3003|3100,3100|3165,3158,1|3|2|3|3|3|4|1|3|1|2|3|1|4|1|1|2|2|4,4|1
326727,KR_5667335218,8,107,Rengar,32,4,8400|8437|8401|8473|8453,8100|8135|8143,5002|5008|5008,200,...,41,7,2,10195,1001|2031|2051,3068|3065,0,3047,1|2|3|2|2|4|2|1|2|3|4|1|3|1|1,32|4
326728,KR_5667335218,9,51,Caitlyn,7,4,8000|8008|8009|9103|8014,8200|8233|8236,5002|5008|5005,200,...,32,15,2,11500,2003|2003|6670,6671|3094|3031,0,3006,1|2|3|1|1|4|1|2|1|2|4|2|2|3|3|4,7|4


In [178]:
champ_games = df.groupby("CHAMPIONNAME")[["GAMEID"]].count().rename(columns={"GAMEID":"GAMES"}).reset_index()

In [176]:
spell_games = df.groupby(["CHAMPIONNAME","SEPLL_LST"])[["WIN"]].count().rename(columns={"WIN":"SPELL_GAMES"}).reset_index()

In [177]:
spell_games[spell_games.CHAMPIONNAME == "Aatrox"]

,CHAMPIONNAME,SEPLL_LST,SPELL_GAMES
0,Aatrox,14|4,8
1,Aatrox,21|4,6
2,Aatrox,32|4,2538
3,Aatrox,32|6,11
4,Aatrox,4|1,23
5,Aatrox,4|3,19
6,Aatrox,6|1,2
7,Aatrox,6|4,207
8,Aatrox,7|4,2
9,Aatrox,7|6,1


In [172]:
spell_win = df.groupby(["CHAMPIONNAME","SEPLL_LST"])[["WIN"]].sum().reset_index()

In [173]:
spell_win[spell_win.CHAMPIONNAME == "Aatrox"]

,CHAMPIONNAME,SEPLL_LST,WIN
0,Aatrox,14|4,3
1,Aatrox,21|4,2
2,Aatrox,32|4,1305
3,Aatrox,32|6,7
4,Aatrox,4|1,12
5,Aatrox,4|3,11
6,Aatrox,6|1,1
7,Aatrox,6|4,107
8,Aatrox,7|4,0
9,Aatrox,7|6,1


In [180]:
pd.merge(spell_games,spell_win, on=["CHAMPIONNAME",'SEPLL_LST'])

,CHAMPIONNAME,SEPLL_LST,SPELL_GAMES,WIN
0,Aatrox,14|4,8,3
1,Aatrox,21|4,6,2
2,Aatrox,32|4,2538,1305
3,Aatrox,32|6,11,7
4,Aatrox,4|1,23,12
...,...,...,...,...
1694,Zyra,32|4,125,64
1695,Zyra,4|1,52,29
1696,Zyra,4|3,1133,578
1697,Zyra,6|4,78,36


In [182]:
spell_win_pick = pd.merge(pd.merge(spell_games,spell_win, on=["CHAMPIONNAME",'SEPLL_LST']),champ_games, on="CHAMPIONNAME")

In [184]:
spell_win_pick["PICK_RATE"] = round(spell_win_pick.SPELL_GAMES / spell_win_pick.GAMES * 100,1)

In [189]:
spell_win_pick["WIN_RATE"] = round(spell_win_pick.WIN / spell_win_pick.SPELL_GAMES * 100,1)

In [193]:
spell_win_pick = spell_win_pick.sort_values("PICK_RATE",ascending=False)

In [195]:
spell_win_pick[spell_win_pick.CHAMPIONNAME == "Aatrox"]

,CHAMPIONNAME,SEPLL_LST,SPELL_GAMES,WIN,GAMES,PICK_RATE,WIN_RATE
2,Aatrox,32|4,2538,1305,2817,90.1,51.4
7,Aatrox,6|4,207,107,2817,7.3,51.7
4,Aatrox,4|1,23,12,2817,0.8,52.2
5,Aatrox,4|3,19,11,2817,0.7,57.9
3,Aatrox,32|6,11,7,2817,0.4,63.6
0,Aatrox,14|4,8,3,2817,0.3,37.5
1,Aatrox,21|4,6,2,2817,0.2,33.3
8,Aatrox,7|4,2,0,2817,0.1,0.0
6,Aatrox,6|1,2,1,2817,0.1,50.0
9,Aatrox,7|6,1,1,2817,0.0,100.0


In [198]:
champ_lst

['Aatrox',
 'Ahri',
 'Akali',
 'Akshan',
 'Alistar',
 'Amumu',
 'Anivia',
 'Annie',
 'Aphelios',
 'Ashe',
 'AurelionSol',
 'Azir',
 'Bard',
 'Blitzcrank',
 'Brand',
 'Braum',
 'Caitlyn',
 'Camille',
 'Cassiopeia',
 'Chogath',
 'Corki',
 'Darius',
 'Diana',
 'DrMundo',
 'Draven',
 'Ekko',
 'Elise',
 'Evelynn',
 'Ezreal',
 'FiddleSticks',
 'Fiora',
 'Fizz',
 'Galio',
 'Gangplank',
 'Garen',
 'Gnar',
 'Gragas',
 'Graves',
 'Gwen',
 'Hecarim',
 'Heimerdinger',
 'Illaoi',
 'Irelia',
 'Ivern',
 'Janna',
 'JarvanIV',
 'Jax',
 'Jayce',
 'Jhin',
 'Jinx',
 'Kaisa',
 'Kalista',
 'Karma',
 'Karthus',
 'Kassadin',
 'Katarina',
 'Kayle',
 'Kayn',
 'Kennen',
 'Khazix',
 'Kindred',
 'Kled',
 'KogMaw',
 'Leblanc',
 'LeeSin',
 'Leona',
 'Lillia',
 'Lissandra',
 'Lucian',
 'Lulu',
 'Lux',
 'Malphite',
 'Malzahar',
 'Maokai',
 'MasterYi',
 'MissFortune',
 'MonkeyKing',
 'Mordekaiser',
 'Morgana',
 'Nami',
 'Nasus',
 'Nautilus',
 'Neeko',
 'Nidalee',
 'Nocturne',
 'Nunu',
 'Olaf',
 'Orianna',
 'Ornn',
 'Pa

In [207]:
tmp = []
for i in champ_lst:
    tmp_lst = []
    tmp_lst.append(spell_win_pick[spell_win_pick.CHAMPIONNAME == i].iloc[0]["CHAMPIONNAME"])
    tmp_lst.append(spell_win_pick[spell_win_pick.CHAMPIONNAME == i].iloc[0]["SEPLL_LST"])
    tmp_lst.append(spell_win_pick[spell_win_pick.CHAMPIONNAME == i].iloc[0]["PICK_RATE"])
    tmp_lst.append(spell_win_pick[spell_win_pick.CHAMPIONNAME == i].iloc[0]["WIN_RATE"])
    
    tmp_lst.append(spell_win_pick[spell_win_pick.CHAMPIONNAME == i].iloc[1]["SEPLL_LST"])
    tmp_lst.append(spell_win_pick[spell_win_pick.CHAMPIONNAME == i].iloc[1]["PICK_RATE"])
    tmp_lst.append(spell_win_pick[spell_win_pick.CHAMPIONNAME == i].iloc[1]["WIN_RATE"])
    
    tmp.append(tmp_lst)

In [212]:
champ_spell = pd.DataFrame(tmp,columns=["CHAMPIONNAME","F_SEPLL","F_PICK_RATE","F_WIN_RATE","S_SEPLL","S_PICK_RATE","S_WIN_RATE"])

In [213]:
champ_spell

,CHAMPIONNAME,F_SEPLL,F_PICK_RATE,F_WIN_RATE,S_SEPLL,S_PICK_RATE,S_WIN_RATE
0,Aatrox,32|4,90.1,51.4,6|4,7.3,51.7
1,Ahri,32|4,28.2,42.3,14|4,27.3,42.3
2,Akali,32|4,94.6,43.0,4|1,2.0,49.2
3,Akshan,4|1,67.8,44.3,4|3,13.4,42.0
4,Alistar,32|4,97.8,54.5,4|3,1.3,57.7
...,...,...,...,...,...,...,...
152,Zed,32|4,94.3,45.8,14|4,3.7,46.7
153,Ziggs,4|3,46.9,55.0,13|4,27.5,52.2
154,Zilean,4|3,70.4,45.8,13|4,12.7,45.5
155,Zoe,32|4,44.3,43.6,14|4,18.0,45.1


In [226]:
def spell_img(row):
    tmp = []
    tmp += list(map(lambda x: spell[x], row["F_SEPLL"].split("|")))
    tmp += [str(row.F_PICK_RATE)]
    tmp += [str(row.F_WIN_RATE)]
    
    tmp += list(map(lambda x: spell[x], row["S_SEPLL"].split("|")))
    tmp += [str(row.S_PICK_RATE)]
    tmp += [str(row.S_WIN_RATE)]
    
    tmp = "|".join(tmp)
    return tmp

In [227]:
champ_spell["SPELL_IMG"] = champ_spell.apply(lambda x: spell_img(x), axis=1)

In [228]:
champ_spell.iloc[0]["SPELL_IMG"]

'https://ddragon.leagueoflegends.com/cdn/11.24.1/img/spell/SummonerSnowball.png|https://ddragon.leagueoflegends.com/cdn/11.24.1/img/spell/SummonerFlash.png|90.1|51.4|https://ddragon.leagueoflegends.com/cdn/11.24.1/img/spell/SummonerHaste.png|https://ddragon.leagueoflegends.com/cdn/11.24.1/img/spell/SummonerFlash.png|7.3|51.7'

# ----------------------------------------------------------------------------------------------------------

# 3. 위 작업의 함수화

In [34]:
def sepll_lst(row):
    if row["SPELL1"] == 4:
        tmp = [row["SPELL2"],row["SPELL1"]]
    elif row["SPELL2"] == 4:
        tmp = [row["SPELL1"],row["SPELL2"]]
    else:
        tmp = [row["SPELL2"],row["SPELL1"]]
        tmp.sort()
    tmp = list(map(str,tmp))
    tmp = "|".join(tmp)
    return tmp

In [7]:
def spell_img(row):
    tmp = []
    tmp += list(map(lambda x: spell[x], row["F_SEPLL"].split("|")))
    tmp += [str(row.F_PICK_RATE)]
    tmp += [str(row.F_WIN_RATE)]
    
    tmp += list(map(lambda x: spell[x], row["S_SEPLL"].split("|")))
    tmp += [str(row.S_PICK_RATE)]
    tmp += [str(row.S_WIN_RATE)]
    
    tmp = "|".join(tmp)
    return tmp

In [35]:
url = "https://ddragon.leagueoflegends.com/cdn/11.24.1/data/ko_KR/summoner.json"
res = requests.get(url).json()
spellId = list(res["data"].keys())
spellkey = list(map(lambda x: res["data"][x]["key"], spellId))
spellurl = list(map(lambda x: "https://ddragon.leagueoflegends.com/cdn/11.24.1/img/spell/"+x+".png", spellId))
spell = dict(zip(spellkey,spellurl))

#df["WIN"] = df.apply(lambda x: 1 if x["WIN"] == "True" else 0 , axis=1)
df["SEPLL_LST"] = df.apply(lambda x: sepll_lst(x), axis=1)

In [36]:
    champ_games = df.groupby("CHAMPIONNAME")[["GAMEID"]].count().rename(columns={"GAMEID":"GAMES"}).reset_index()
    spell_games = df.groupby(["CHAMPIONNAME","SEPLL_LST"])[["WIN"]].count().rename(columns={"WIN":"SPELL_GAMES"}).reset_index()
    spell_win = df.groupby(["CHAMPIONNAME","SEPLL_LST"])[["WIN"]].sum().reset_index()
    spell_win_pick = pd.merge(pd.merge(spell_games,spell_win, on=["CHAMPIONNAME",'SEPLL_LST']),champ_games, on="CHAMPIONNAME")
    spell_win_pick["PICK_RATE"] = round(spell_win_pick.SPELL_GAMES / spell_win_pick.GAMES * 100,1)
    spell_win_pick["WIN_RATE"] = round(spell_win_pick.WIN / spell_win_pick.SPELL_GAMES * 100,1)
    spell_win_pick = spell_win_pick.sort_values("PICK_RATE",ascending=False)

    tmp = []
    for i in champ_lst:
        tmp_lst = []
        tmp_lst.append(spell_win_pick[spell_win_pick.CHAMPIONNAME == i].iloc[0]["CHAMPIONNAME"])
        tmp_lst.append(spell_win_pick[spell_win_pick.CHAMPIONNAME == i].iloc[0]["SEPLL_LST"])
        tmp_lst.append(spell_win_pick[spell_win_pick.CHAMPIONNAME == i].iloc[0]["PICK_RATE"])
        tmp_lst.append(spell_win_pick[spell_win_pick.CHAMPIONNAME == i].iloc[0]["WIN_RATE"])

        tmp_lst.append(spell_win_pick[spell_win_pick.CHAMPIONNAME == i].iloc[1]["SEPLL_LST"])
        tmp_lst.append(spell_win_pick[spell_win_pick.CHAMPIONNAME == i].iloc[1]["PICK_RATE"])
        tmp_lst.append(spell_win_pick[spell_win_pick.CHAMPIONNAME == i].iloc[1]["WIN_RATE"])

        tmp.append(tmp_lst)

    champ_spell = pd.DataFrame(tmp,columns=["CHAMPIONNAME","F_SEPLL","F_PICK_RATE","F_WIN_RATE","S_SEPLL","S_PICK_RATE","S_WIN_RATE"])

    champ_spell["SPELL_IMG"] = champ_spell.apply(lambda x: spell_img(x), axis=1)



In [11]:
df = df[df['SKILL_BUILD'].notna()]
# 챔피언 리스트 담아두기
champ_lst = list(df.groupby("CHAMPIONNAME").count().reset_index()["CHAMPIONNAME"])

In [12]:
champ_spell = champ_spell_win_pick_rate(df)

In [41]:
champ_spell[champ_spell.CHAMPIONNAME == "Yuumi"]

,CHAMPIONNAME,F_SEPLL,F_PICK_RATE,F_WIN_RATE,S_SEPLL,S_PICK_RATE,S_WIN_RATE,SPELL_IMG
150,Yuumi,3|13,53.7,52.5,3|14,28.8,52.1,https://ddragon.leagueoflegends.com/cdn/11.24....
